# Хроматическое число R^4

### 1. Описание алгоритма

Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем словари, ключами которых являются координаты точек s, а значениями - минимальное хроматическое число (dict_det), максимальное запрещенное расстоянием (dict_dist), список матриц (dict_s), соответствующих данной точке s, хроматическому числу и запрещенному расстоянию
- составляем датафрейм (df), обьединяя все эти словари
- составляем список матриц для каждого подходящего хроматического числа (если запрещенное расстояние больше 1)

- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного

- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- edge_central - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные


- list_neigh_points - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника)
- dict_point_edge - словарь: ключ - координаты соседней точки, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
- index_point_edge - индекс грани в списке граней центрального многогранника (edge_central_coords)
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)
- point_face_norm - точки, принадлежащие грани и лежащие на векторе нормали (середина вектора, соединяющего центры)

### 2. Подключение библиотек

In [39]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from copy import deepcopy
from collections import defaultdict

from itertools import combinations
from math import prod
from sympy import factorint

import networkx as nx
import time

### 3. Построение многогранника Вороного

In [2]:
class VoronoiPolyhedra:
    def __init__(self):
        self.grid
        self.central
        ...
    
    def build():
        ...

In [3]:
grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

coords4 = [] # координаты центров

for var in itertools.product(range(-3, 3), repeat=4): # меньше 3 нельзя - тк при поиске центр региона отбрасываем незаконченные регионы
    coords4.append((grid.T).dot(var).tolist())
    
# строим диаграмму Вороного
vor4 = Voronoi(coords4)

может быть убрать, тк regions_finite не используется дальше 

In [5]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for region in vor4.regions:
    if -1 not in region:
        list_coord_reg = []
        for vert in region:
            list_coord_reg.append(vor4.vertices[vert])
        vor4.regions_finite.append(list_coord_reg)
        count += 1

vor4.regions_finite.pop(0)  # удаляем первый пустой регион 
      

[]

### 4. Построение центрального региона

может быть убрать тк далее не используем

In [6]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for region in vor4.regions:
    len_r = len(region)
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество таких регионов =', count_r)

количество вершин = 24 количество таких регионов = 257


In [8]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник central

sum_dist_min = 1000

for region in vor4.regions[1:]:
    
    len_r = len(region)

    if -1 in region: continue # регион неограничен
        
    length = 0
    
    # для каждого региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
    for j in range(len_r):
        length += distance.euclidean(vor4.vertices[region[j]], [0.0, 0.0, 0.0, 0.0])

    if length < sum_dist_min:
        sum_dist_min = length
        v_min = region

# ищу индекс минимального региона
v_min = next((i for i, arr in enumerate(vor4.regions) if np.array_equal(arr, v_min)), None)

print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

central = vor4.vertices[vor4.regions[v_min]]
vor4.central = vor4.vertices[vor4.regions[v_min]]


суммарное расстояние = 24.0 индекс центрального региона = 393


 наверное эту колонку надо удалить - не используется дальше

In [9]:
# нахожу длину ребра центрального политопа

d_edge_min = 100
count = 0
for i in range(0, len(central)-1):
    for j in range(i + 1, len(central)):
        d_edge = distance.euclidean(central[i], central[j])
        if d_edge_min >= d_edge:
            d_edge_min = d_edge
            count += 1
            
d_edge_min, count
                           

(1.0, 96)

In [10]:
# нашли индекс точки(центра) центрального многогранника

centr_index = coords4.index([0.0, 0.0, 0.0, 0.0])
coords4[centr_index]

[0.0, 0.0, 0.0, 0.0]

In [11]:
# составляем список центров многогранников, граничащих с центральным
# индексы координат в coords4

list_neigh_points = [] # список центров многогранников, граничащих с центральным
vor4.list_neigh_points = []

for pair in vor4.ridge_points:

    if centr_index not in pair: continue

    if pair[0] == centr_index:
        list_neigh_points.append(coords4[pair[1]])
        vor4.list_neigh_points.append(coords4[pair[1]])
        #print(coords4[pair[1]])
    else:
        list_neigh_points.append(coords4[pair[0]])
        vor4.list_neigh_points.append(coords4[pair[0]])
    

In [13]:
# трехмерные ячейки 

edge_central = [] # индексы координат вершин 3х мерных граней 


# перебирам 3х мерные грани 
for face in vor4.ridge_vertices:

    count = 0

    # перебираем вершины каждой 3хмерной грани
    for vert in face:
        
        # если координата в списке координат центрального региона, увеличиваем count на 1
        if vert in vor4.regions[v_min]:
            count += 1 
            
    # если count = количеству вершин в многограннике, то мы нашли 
    if count == len(face):
        edge_central.append(face)
        
# перевожу индексы вершин в координаты

edge_central_coords = [] # координаты вершин 3х мерных граней

for face in edge_central:
    r = []
    for vert in face:
        r.append(vor4.vertices[vert])
    edge_central_coords.append(np.array(r))
    

In [15]:
# ищем для каждого центра ближайшую трехмерную грань
# берем каждую точку из списка соседних центральных точек и ищем расстояние от нее до каждой 3х мерной грани
# суммируем и берем минимальное ссумарное расстояние. это и будет 3х мерная грань между точкой (0, 0, 0, 0)
# точкой из списка
# и вектор, соединяющий центры будет вектором нормали к данной 3х мерной грани


dict_point_edge = {} # словарь: ключ - соседняя точка, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
index_point_edge = 0 # индекс грани в списке граней центрального многогранника (edge_central_coords[)
list_ridge_edge = [] # список граней, в том же порядке, что и соседние точки (list_neigh_points)


# перебираем все соседние центры

for point in list_neigh_points:
    
    min_dist_point_edge = 1000

    # для каждой 3х мерной грани   
    for edge in edge_central_coords:
        
        dist_point_edge = 0
        
        # для каждой вершины их 3х мерной грани считаем растояние до соседнего центра и суммируем
        for coord in edge:
            
            dist_point_edge += distance.euclidean(coord, point)
            
        # если суммарное растояние меньше минимального, записываем новое суммарное растояние и индекс грани    
        if min_dist_point_edge > dist_point_edge:
            min_dist_point_edge = dist_point_edge
            point_edge = edge

    v_min = next((i for i, arr in enumerate(vor4.regions) if np.array_equal(arr, v_min)), None)


    dict_point_edge[tuple(point)] = point_edge
    list_ridge_edge.append(edge)
    


In [16]:
# Находим коэффициенты плоскости, проходящей через точки coord1, coord2, coord3
# Система уравнений: a*x + b*y + c*z + d*w + e = 0

def hyperplane_coefficients(coord1, coord2, coord3):
    vec = np.vstack((coord1, coord2, coord3))
    vec = vec.T

    norm1 = -np.linalg.det(vec[1:, :])
    norm2 = np.linalg.det(vec[[0, 2, 3], :])
    norm3 = -np.linalg.det(vec[[0, 1, 3], :])
    norm4 = np.linalg.det(vec[[0, 1, 2], :])
    normal = np.array([norm1, norm2, norm3, norm4])
    e = -np.dot(normal, coord1)
    
    return np.array([norm1, norm2, norm3, norm4, e])

# Проверяем, лежат ли все точки по одну сторону от плоскости
def is_on_same_side(hyperplane, points):

    signs = []
    for point in points:
        value = np.dot(hyperplane[:-1], point) + hyperplane[-1]
        signs.append(np.sign(value))

    # Если все знаки одинаковы, точки лежат по одну сторону
    return all(s == signs[0] for s in signs)

# является ли пара вершин ребром (строим плоскость через)
def is_edge(vertices, i, j):

    n = len(vertices)

    # если есть хоть одна точка k, с которой i и j образуют плоскость, относительно которой все остальняе точки 
    # многогранника лежат с одной стороны от нее.
    for k in range(n):
        if k == i or k == j: 
            continue

        # Строим плоскость через вершины i, j, k
        hyperplane = hyperplane_coefficients(vertices[i], vertices[j], vertices[k])

        # Проверяем положение всех остальных вершин
        other_points = [vertices[l] for l in range(n) if l != i and l != j and l != k]
        
        if is_on_same_side(hyperplane, other_points):
            return True
        
    return False

# ищем ребра, перебиоая все пары вершин
def find_edges(vertices):
    edges = []
    n = len(vertices)
    for i in range(n):
        for j in range(i + 1, n):
            if is_edge(vertices, i, j):
                edges.append((vertices[i], vertices[j]))
    return edges


In [17]:
def find_2d_faces(vertices):
    '''
    Находит все двумерные грани трёхмерного выпуклого многогранника в 4D.
    :param vertices: Массив вершин многогранника.
    :return: Список двумерных граней (каждая грань — это список индексов вершин).
    '''
    n = len(vertices)
    faces = []

    # Перебираем все возможные тройки вершин для построения плоскости
    for comb in combinations(range(n), 3):
        coord1, coord2, coord3 = vertices[comb[0]], vertices[comb[1]], vertices[comb[2]]
        
        # Находим коэффициенты плоскости, проходящей через coord1, coord2, coord3
        hyperplane = hyperplane_coefficients(coord1, coord2, coord3)

        # Проверяем, что все остальные вершины лежат по одну сторону от плоскости
        other_points = [vertices[l] for l in range(n) if np.any(vertices[l] != coord1) and \
                        np.any(vertices[l] != coord2)\
                        and np.any(vertices[l] != coord3)]
        
        if is_on_same_side(hyperplane, other_points):

            # Находим все вершины, лежащие на этой плоскости
            face_vertices = []
            for i in range(n):
                point = vertices[i]
                value = np.dot(hyperplane[:-1], point) + hyperplane[-1]
                if np.isclose(value, 0, atol=1e-8):  # Если точка на плоскости
                    face_vertices.append(i)

            # Добавляем грань, если она содержит хотя бы 3 вершины
            if len(face_vertices) >= 3:

                # Сортируем порядок вершин
                face_vertices_sorted = sorted(face_vertices)
                faces.append(face_vertices_sorted)
      
    # Удаляем дубликаты граней
    unique_faces = [] # уникальные наборы индексов 2d граней
    faces_coord = [] # список уникальных наборов координат 2d граней

    for face in faces:
        if face not in unique_faces:
            list_f = []
            for index in face:
                list_f.append(vertices[index])
            
            faces_coord.append(list_f)
            unique_faces.append(face)
    
    return faces_coord

In [18]:
# строим список 2d граней 3d граней
list_faces = []
for face in edge_central_coords:
    list_faces.append(find_2d_faces(face)) 

In [19]:
# диаметр многогранника (ищем максимальное расстояние между вершинами) - 2

max_len = 2 * distance.euclidean(np.array([0, 0, 0, 0]), central[1])
max_len

2.0

In [22]:
# нормируем вектора нормали
# ищем точки, принадлежащие граням и находящиеся на векторах нормали

v_norm = [] # здесь будем хранить нормированные вектора нормали
point_face_norm = [] #точки, принадлежащие граням и находящиеся на векторах нормали

for vec in list_neigh_points:
    v_norm.append(np.array(vec / np.linalg.norm(vec)))
    point_face_norm = np.array(vec) / 2
        

In [23]:
vertex_to_faces = [] # собой список списков 3D-граней для каждой вершины 
                        # (где индексы граней берутся из edge_central_coords)

for vertex in central:
    
    tamp_list = [] # список граней для текущей вершины
    
    for face in range(len(edge_central_coords)):

        if np.any(np.all(edge_central_coords[face] == vertex, axis=1)):
            tamp_list.append(face)
            
    vertex_to_faces.append(tamp_list)


In [24]:
# строим триангуляцию

delaunay = Delaunay(central)

### 5. Создание классов граней и ребер центрального многогранника

In [25]:
class Edge2D:
    def __init__(self, vertex1, vertex2, face_center):
        '''
        Инициализация ребра.
        :vertex1: Координаты первой вершины ребра.
        :vertex2: Координаты второй вершины ребра.
        :face_center: Координаты центра грани.
        '''
        self.vertex1 = np.array(vertex1)
        self.vertex2 = np.array(vertex2)
        self.face_center = np.array(face_center)

        # Вычисляем середину ребра
        self.center = (self.vertex1 + self.vertex2) / 2

        # Вычисляем нормаль к ребру
        #edge_vector = self.vertex2 - self.vertex1
        self.normal = self.center - self.face_center  # Перпендикулярный вектор
        self.normal = self.normal / np.linalg.norm(self.normal)  # Нормализация
        
        self.bias = self.normal @ vertex1

        # Проверяем направление нормали (она должна быть направлена от центра грани)
        vector_to_center = self.face_center - self.center
        if np.dot(self.normal, vector_to_center) > 0:
            self.normal = -self.normal  # Меняем направление нормали

    def __repr__(self):
        return f"Edge2D(vertex1={self.vertex1}, vertex2={self.vertex2}, normal={self.normal}, \
                midpoint={self.center})"

In [26]:
class Face2D:
    def __init__(self, vertices, polyhedron_center):
        '''
        Инициализация 2D грани многогранника.
        '''
        self.vertices = vertices  # Координаты вершин грани
        self.parent_center = polyhedron_center # центр 3х мерной грани
        self.center = self._calculate_center()  # Центр грани
        self.normal = self._calculate_normal()  # Нормаль к грани
        self.edges = self._calculate_edges()  # ребра в 2х мерном многограннике
        self.bias = self.normal @ vertices[0]

    def _calculate_center(self):
        '''
        Вычисляет центр грани как среднее арифметическое координат вершин.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    def _calculate_normal(self):

        # Вычисляем нормаль к грани как вектор из центра 3х мерного многогранника в центр 2х мерной грани
        normal_3d = np.array(self.center) - np.array(self.parent_center)

        # Нормализуем нормаль
        normal_3d = normal_3d / np.linalg.norm(normal_3d)

        return normal_3d

    def _calculate_edges(self):
        
        # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
        list_edges = []
        
        for i in range(len(self.vertices)-1):
            for j in range(i+1, len(self.vertices)):
                
                d = distance.euclidean(self.vertices[i], self.vertices[j])
                
                if d == 1:
                    vertex1 = self.vertices[i]
                    vertex2 = self.vertices[j]  
                    edge = Edge2D(vertex1, vertex2, self.center)
                    list_edges.append(edge)
        
        return list_edges

    
    def __repr__(self):
        '''
        Возвращает строковое представление грани.
        '''
        return (f"Face2D(vertices={self.vertices}), center={self.center}, "
                f"normal={self.normal}")

In [44]:
#class Polyhedron4D:
class Face3D:
    def __init__(self, vertices, faces_list, normal):
        '''
        Инициализация многогранника в 4-мерном пространстве.
        :vertices: Список координат вершин многогранника .
        :normal: Вектор нормали к многограннику.
        '''
        self.vertices = vertices  # Координаты вершин
        self.normal = normal  # Нормаль к многограннику
        self.faces_list = faces_list  # Грани многогранника
        self.center = self._calculate_center()  # Центр многогранника
        self.faces = self._find_faces() # грани многогранника (как экземпляр класса)
        self.bias = normal @ vertices[0] # смещение

    def _calculate_center(self):
        '''
        Вычисляет центр многогранника как среднее арифметическое координат вершин.
        :return: Координаты центра.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    
    def _find_faces(self):
        '''
        Находит грани многогранника и создаёт объекты Face2D.
        :return: Список объектов Face2D.
        '''
        
        faces = []

        for face_vertices in self.faces_list:
            face = Face2D(face_vertices, self.center)
            faces.append(face)

        return faces
    

    def __repr__(self):
        '''
        Возвращает строковое представление многогранника.
        '''
        return (f"Face3D(vertices={self.vertices}, normal={self.normal}, "
                f"center={self.center}, faces={self.faces})")


In [45]:
# расчитываем все грани центрального многогранника

polyhedrons = []  # Список для хранения всех объектов

for face_index in range(len(edge_central_coords)):
    polyhedron = Face3D(edge_central_coords[face_index], list_faces[face_index], v_norm[face_index])
    polyhedrons.append(polyhedron)
    

### 6. Расстояние от точки s до центрального многогранника

In [30]:
# через минимальное расстояние до вершины

def dist_to_s(polyhedrons, s):
    
    min_dist_to_pol = float('inf')
    index_proj = -1
    min_dist_vert_to_s = float('inf')
    min_vert = -1

    # находим ближайшую вершину к точке s
    
    for index in range(24):

        dist_vert_to_s = distance.euclidean(s, central[index])

        if dist_vert_to_s < min_dist_vert_to_s:
            min_dist_vert_to_s = dist_vert_to_s
            min_vert = index

    # находим расстояние и проекцию на центральный многогранник
    
    for i in vertex_to_faces[min_vert]: # рассматриваем только грани,которым принадлежит ближайшая в точке s вершина

        d0 = polyhedrons[i].normal @ (s - polyhedrons[i].center)
        coord0 = s - (d0 + polyhedrons[i].bias)* polyhedrons[i].normal
        simplex = delaunay.find_simplex(coord0)
        
        if simplex != -1: # если проекция принадлежит центральному многораннику

            dist = abs(d0)
            coords_to_central = coord0

        else:
            for face2d in polyhedrons[i].faces: #cycle

                d1 = face2d.normal @ (coord0 - face2d.center)
                coord1 = coord0 - (d1 + face2d.bias)* polyhedrons[i].normal
                simplex = delaunay.find_simplex(coord0)

                if simplex != -1: # если проекция принадлежит центральному многораннику
                    dist = distance.euclidean(s, coord1, dtype = 'float')
                    coords_to_central = coord1

                else:
                    for edge in face2d.edges:
                        d2 = edge.normal @ (coord1 - edge.center)
                        coord2 = coord1 - (d2 + edge.bias) * edge.normal
                        simplex = delaunay.find_simplex(coord0)

                        if simplex != -1: # если проекция принадлежит центральному многораннику
                            dist = distance.euclidean(s, coord2, dtype = 'float')
                            coords_to_central = coord2

                        else:
                            d3 = distance.euclidean(s, edge.vertex1)
                            d4 = distance.euclidean(s, edge.vertex2)

                            if d3 < d4:
                                dist = d3
                                coords_to_central = edge.vertex1
                            else:
                                dist = d4
                                coords_to_central = edge.vertex2

        if min_dist_to_pol > dist:
            
            min_dist_to_pol = dist
            coords_proj = coords_to_central
            index_proj = i
        
        # если расстояние до какой-либо грани < 1, то дальше не считаем
        if  min_dist_to_pol < 1:

            return min_dist_to_pol, coords_proj, index_proj

        
    return min_dist_to_pol, coords_proj, index_proj

In [31]:
polyhedrons[9].faces[0].edges[0].center

array([ 0.5, -0.5,  0. ,  0.5])

In [32]:
min_d = 1.5

### 7. LLL-базис (функция)

In [33]:
def gram_schmidt(v):

    w_1 = copy.deepcopy(v[0])
    w_n = copy.deepcopy(w_1)
    w_array = [deepcopy(w_n)]
    
    for n in range(1, len(v)):
        v_n = copy.deepcopy(v[n])
        w_n = copy.deepcopy(v_n)

        for j in range(n):
            w_j = deepcopy(w_array[j])
            if not any(w_j):
                continue
            w_n -= np.dot(v_n, w_j) / np.dot(w_j, w_j) * w_j
        
        w_array += [w_n]
          
    return w_array

In [34]:
d = 0.75

def mu(b_i, b_j):
    return np.dot(b_i, b_j) / np.dot(b_j, b_j) if np.dot(b_j, b_j) != 0 else 0


def LLL(l_basis):

    ortho = gram_schmidt(l_basis)

    k = 1
    n = len(ortho)

    while k < n:
        for j in range(k - 1, -1, -1):
            proj = mu(l_basis[k], ortho[j])
            if abs(proj) > 1/2:
                l_basis[k] -= l_basis[j] * round(proj)
                ortho = copy.deepcopy(gram_schmidt(l_basis))
            #print(l_basis, 111)

        if np.dot(ortho[k], ortho[k]) >= (d - (mu(l_basis[k], ortho[k-1]))**2) * (np.dot(ortho[k-1], ortho[k-1])):
            #print(l_basis, 222)
            k += 1

        else:
            s = copy.deepcopy(l_basis[k-1])
            l_basis[k-1] = copy.deepcopy(l_basis[k])
            l_basis[k] = copy.deepcopy(s)
            ortho = gram_schmidt(l_basis)
            k = max(k-1, 1)
            #print(l_basis, 333)

    return l_basis

### 8. Поиск подрешеток

In [35]:
# генерация комбинаций

def generate_factor_combinations(factors):
    if not factors:
        return [[]]
    
    first, *rest = factors
    rest_combinations = generate_factor_combinations(rest)
    
    result = []
    for comb in rest_combinations:
        result.append([first] + comb)
        result.append([first * (comb[0] if comb else 1)] + (comb[1:] if len(comb) > 1 else []))
    
    return result

In [36]:
# находит все возможные разложения числа на простые множители

def compute_factorizations(n):
    
    # находим простые множители числа
    
    factors = [] # список простых множителей
    
    for prime, exp in factorint(n).items():
        factors.extend([prime] * exp)
    
    combinations = []
    combinations = generate_factor_combinations(factors) # генерируем список всех возможных комбинаций
    
    unique = set(tuple(sorted(comb)) for comb in combinations) # убираем дубликаты и сортируем
    unique_combinations = [list(comb) for comb in sorted(unique)]
    
    return unique_combinations
    


In [37]:
list_all_factorizations = compute_factorizations(54)
list_all_factorizations

[[2, 3, 3, 3], [2, 3, 9], [2, 27], [3, 3, 6], [3, 18], [6, 9], [54]]

In [ ]:
factorint(54)

{2: 1, 3: 3}

In [38]:
def pad_lists_with_ones(list_of_lists):
    # Проходим по каждому списку в основном списке
    for i in range(len(list_of_lists)):
        # Если длина списка меньше 4
        if len(list_of_lists[i]) < 4:
            # Добавляем в начало столько единиц, сколько не хватает до 4
            list_of_lists[i] = [1] * (4 - len(list_of_lists[i])) + list_of_lists[i]
    return list_of_lists


In [42]:
list_mat = []
list_lll = []
list_s1 = []
list_dist = []

for det_mat in range(54, 57):

    print(det_mat)

    list_all_factorizations = compute_factorizations(det_mat)
    list_diag_el = pad_lists_with_ones(list_all_factorizations)

    for diag_el in list_diag_el:
        mat = np.array([[diag_el[0], 0, 0, 0], 
                        [0, diag_el[1], 0, 0], 
                        [0, 0, diag_el[2], 0], 
                        [0, 0, 0, diag_el[3]]], float)
        max_num_col3 = diag_el[3]
        max_num_col2 = diag_el[2]
        max_num_col1 = diag_el[1]
        #max_num_col0 = diag_el[0]
        print(max_num_col1, max_num_col2, max_num_col3)

        for i1 in range(max_num_col3):
            mat[2][3] = i1
            print(i1)
            for i2 in range(max_num_col3):
                mat[1][3] = i2
                for i3 in range(max_num_col2):
                    mat[1][2] = i3
                    for i4 in range(max_num_col3):
                        mat[0][3] = i4
                        for i5 in range(max_num_col2):
                            mat[0][2] = i5
                            for i6 in range(max_num_col1):
                                mat[0][1] = i6
                                
                                sub_grid = np.dot(mat, grid)
                                basis = LLL(sub_grid)

                                # ищем самый короткий вектор
                                min_dist = 100
                                index = 0
                                for i in range(4):
                                    d_vec = np.linalg.norm(basis[i])
                                    if d_vec < min_dist:
                                        min_dist = d_vec
                                        index = i

                                # определяем точку s как половину длины самого короткого базисного вектора
                                if min_dist > min_d:
                                    s1 = basis[index] * 0.5
                                    #print(mat, s1)
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1)
                                    
                                    if dist_s1 > 1:
                                        print(dist_s1, s1, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1)

54
3 3 3
0
1
2
2 3 9
0
1.0606601717798212 [1.5 1.  0.  0.5] [[1. 1. 0. 6.]
 [0. 2. 0. 1.]
 [0. 0. 3. 0.]
 [0. 0. 0. 9.]]
1
2
3
4
5
6
7
8
1 2 27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1.0606601717798212 [ 1.5 -0.5  1.   0. ] [[ 1.  0.  1.  9.]
 [ 0.  1.  1. 24.]
 [ 0.  0.  2. 15.]
 [ 0.  0.  0. 27.]]
16
17
18
19
20
21
22
23
24
25
26
3 3 6
0
1
2
3
4
5
1.0606601717798212 [-1.  -1.5  0.   0.5] [[1. 2. 2. 0.]
 [0. 3. 0. 5.]
 [0. 0. 3. 5.]
 [0. 0. 0. 6.]]
1 3 18
0
1.0606601717798212 [1.5 0.5 0.  1. ] [[ 1.  0.  2. 12.]
 [ 0.  1.  0.  2.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [1.  0.  0.5 1.5] [[ 1.  0.  2. 15.]
 [ 0.  1.  2.  4.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [-1.5  0.5  0.  -1. ] [[ 1.  0.  2. 12.]
 [ 0.  1.  2. 10.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [1.  0.  0.5 1.5] [[ 1.  0.  2. 15.]
 [ 0.  1.  0. 11.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1
2
3
4
5
6
7
8
9
10
11
12
1.0606601717798212 [ 0.5 -1.5  0.   1. ] [[ 1.  0. 

In [ ]:
import time

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 54
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
start - end